In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
spark = SparkSession.builder.appName("SparkSQL Movie Data Analysis").enableHiveSupport().getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
hdfs_path = '/tmp/datasets/movies.csv'
movies = spark.read.format("csv").option("multiline", "true").option("header", "true").load(hdfs_path)
movies.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- movieId: string (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

In [30]:
movies.show(15)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
+-------+--------------------+--------------------+
only showing

In [31]:
# define Schema
movies_schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("genres", StringType(), True)
])
movies = spark.read.format("csv").option("multiline", "true").option("header", "true").schema(movies_schema).load(hdfs_path)
movies.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

In [32]:
hdfs_path_r = '/tmp/datasets/ratings.csv'
ratings_schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", FloatType(), True),
    StructField("timestamp", IntegerType(), True)
])

ratings = spark.read.format("csv").option("header", "true").option("multiline", "true").schema(ratings_schema).load(hdfs_path_r)
ratings.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)
 |-- timestamp: integer (nullable = true)

In [33]:
ratings.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows

In [34]:
ratings = ratings.withColumn("timestamp", to_timestamp(ratings["timestamp"]))
ratings.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      1|   4.0|2000-07-30 18:45:03|
|     1|      3|   4.0|2000-07-30 18:20:47|
|     1|      6|   4.0|2000-07-30 18:37:04|
|     1|     47|   5.0|2000-07-30 19:03:35|
|     1|     50|   5.0|2000-07-30 18:48:51|
+------+-------+------+-------------------+
only showing top 5 rows

In [35]:
hdfs_path_t = '/tmp/datasets/tags.csv'
tags_schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("tag", StringType(), True),
    StructField("timestamp", IntegerType(), True)
])

tags = spark.read.format("csv").option("header", "true").option("multiline", "true").schema(tags_schema).load(hdfs_path_t)
tags.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- tag: string (nullable = true)
 |-- timestamp: integer (nullable = true)

In [36]:
tags = tags.withColumn("timestamp", to_timestamp(tags["timestamp"]))
tags.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------+---------------+-------------------+
|userId|movieId|            tag|          timestamp|
+------+-------+---------------+-------------------+
|     2|  60756|          funny|2015-10-24 19:29:54|
|     2|  60756|Highly quotable|2015-10-24 19:29:56|
|     2|  60756|   will ferrell|2015-10-24 19:29:52|
|     2|  89774|   Boxing story|2015-10-24 19:33:27|
|     2|  89774|            MMA|2015-10-24 19:33:20|
+------+-------+---------------+-------------------+
only showing top 5 rows

# 1. Show the aggregated number of ratings per year

In [37]:
movies.createOrReplaceTempView("MOVIES")
ratings.createOrReplaceTempView("RATINGS")
tags.createOrReplaceTempView("TAGS")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
query = """SELECT year(timestamp) AS YEAR, count(rating) AS rating FROM RATINGS
        GROUP BY YEAR ORDER BY 1 desc"""

output = spark.sql(query)
output.show()

output.coalesce(1).write.mode("overwrite").format('csv').option('header', 'true').option('delimiter', ',').save('/tmp/output_data/Movie_Data_Analysis/agg_ratings_per_year.csv')
print("Write Successful")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+
|YEAR|rating|
+----+------+
|2018|  6418|
|2017|  8198|
|2016|  6703|
|2015|  6616|
|2014|  1439|
|2013|  1664|
|2012|  4656|
|2011|  1690|
|2010|  2301|
|2009|  4158|
|2008|  4351|
|2007|  7114|
|2006|  4059|
|2005|  5813|
|2004|  3279|
|2003|  4014|
|2002|  3478|
|2001|  3922|
|2000| 10061|
|1999|  2439|
+----+------+
only showing top 20 rows

Write Successful

# 2. Show the average monthly number of ratings

In [39]:
query = """ SELECT year(timestamp) AS Year,month(timestamp) AS Month, ROUND(AVG(rating),5) AS Average_Rating
FROM RATINGS
GROUP BY 1, 2
ORDER BY 1 DESC, 2 DESC"""
output = spark.sql(query)
output.show()

output.coalesce(1).write.mode("overwrite").format('csv').option('header', 'true').option('delimiter', ',').save('/tmp/output_data/Movie_Data_Analysis/agg_ratings_per_year_per_month.csv')
print("Write Successful")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+--------------+
|Year|Month|Average_Rating|
+----+-----+--------------+
|2018|    9|       3.56871|
|2018|    8|       3.55776|
|2018|    7|       4.01024|
|2018|    6|       3.97971|
|2018|    5|       2.95163|
|2018|    4|          3.75|
|2018|    3|       3.78682|
|2018|    2|       2.73867|
|2018|    1|       3.41947|
|2017|   12|       3.26119|
|2017|   11|       3.65217|
|2017|   10|       3.52444|
|2017|    9|       3.68278|
|2017|    8|       4.07692|
|2017|    7|       4.05294|
|2017|    6|       2.95942|
|2017|    5|       3.48018|
|2017|    4|       3.62622|
|2017|    3|         3.051|
|2017|    2|       2.75476|
+----+-----+--------------+
only showing top 20 rows

Write Successful

# 3. Show the rating levels distribution

In [40]:
query = """
WITH t1 AS (
  SELECT rating,
  CASE
    WHEN rating > 0 AND rating < 1 THEN "0 - 0.9"
    WHEN rating >= 1 AND rating < 2.5 THEN "1 - 2.4"
    WHEN rating >= 2.5 AND rating < 3.5 THEN "2.5 - 3.4"
    WHEN rating >= 3.5 THEN "3.5 - 5.0"
  END AS Rating_Range
  FROM RATINGS
),
t2 AS (
SELECT Rating_Range, count(rating) as Count
FROM t1
GROUP BY 1
ORDER BY 1)

SELECT Rating_Range,count, ROUND(count*100/sum(count)over(),1) Percentage FROM t2

"""

output = spark.sql(query)
output.show()

output.coalesce(1).write.mode("overwrite").format("csv").option('header', 'true').option('delimiter', ',').save('/tmp/output_data/Movie_Data_Analysis/rating_levels_distribution.csv')
print("Write Successful!")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+-----+----------+
|Rating_Range|count|Percentage|
+------------+-----+----------+
|     0 - 0.9| 1370|       1.4|
|     1 - 2.4|12153|      12.1|
|   2.5 - 3.4|25597|      25.4|
|   3.5 - 5.0|61716|      61.2|
+------------+-----+----------+

Write Successful!

# 4. Show the 18 movies that are tagged but not rated

In [41]:
query = """WITH t1 AS
(SELECT DISTINCT T.movieId FROM TAGS T
LEFT OUTER JOIN RATINGS R
ON T.movieId = R.movieId
WHERE R.movieId IS NULL)

SELECT M.title FROM
MOVIES M INNER JOIN t1
ON M.movieId = t1.movieId
ORDER BY 1
"""

output = spark.sql(query)
output.show()

output.coalesce(1).write.mode("overwrite").format('csv').option('header', 'true') .option('delimiter', ',').save('/tmp/output_data/Movie_Data_Analysis/movies_tagged_but_not_rated.csv')
print("Write Successful!")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|               title|
+--------------------+
|Browning Version,...|
|Call Northside 77...|
|  Chalet Girl (2011)|
|  Chosen, The (1981)|
|Color of Paradise...|
|For All Mankind (...|
|I Know Where I'm ...|
|In the Realms of ...|
|Innocents, The (1...|
|Mutiny on the Bou...|
|      Niagara (1953)|
|Parallax View, Th...|
|        Proof (1991)|
|Road Home, The (W...|
|Roaring Twenties,...|
|      Scrooge (1970)|
|This Gun for Hire...|
|Twentieth Century...|
+--------------------+

Write Successful!

# 5. Show the movies that have rating but not tagged

In [42]:
query = """WITH t1 AS
(SELECT DISTINCT R.movieId FROM RATINGS R
LEFT OUTER JOIN TAGS T
ON T.movieId = R.movieId
WHERE T.movieId IS NULL)

SELECT M.title FROM
MOVIES M INNER JOIN t1
ON M.movieId = t1.movieId
ORDER BY 1
"""

output = spark.sql(query)
output.show()

output.coalesce(1).write.mode("overwrite").format('csv').option('header', 'true') .option('delimiter', ',').save('/tmp/output_data/Movie_Data_Analysis/movies_rated_but_not_tagged.csv')
print("Write Successful!")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|               title|
+--------------------+
|          '71 (2014)|
|'Hellboy': The Se...|
|'Round Midnight (...|
| 'Salem's Lot (2004)|
|'Til There Was Yo...|
|'Tis the Season f...|
|  'burbs, The (1989)|
|'night Mother (1986)|
|*batteries not in...|
|...All the Marble...|
|00 Schneider - Ja...|
|   1-900 (06) (1994)|
|           10 (1979)|
|10 Cent Pistol (2...|
|10 Items or Less ...|
|     10 Years (2011)|
|    10,000 BC (2008)|
|    100 Girls (2000)|
|  100 Streets (2016)|
|101 Dalmatians II...|
+--------------------+
only showing top 20 rows

Write Successful!

# 6. Focusing on the rated untagged movies with more than 30 user ratings, show the top 10 movies in terms of average rating and number of ratings

In [43]:
query = """WITH t1 AS (
  SELECT R.movieId, COUNT(R.rating) AS total_count, AVG(R.rating) AS average_rating
  FROM RATINGS R
  LEFT OUTER JOIN TAGS T ON R.movieId = T.movieId
  WHERE T.tag IS NULL
  GROUP BY R.movieId
  HAVING COUNT(R.rating) > 30
  ORDER BY total_count DESC
),

t2 AS (
  SELECT M.title, t1.movieId, t1.average_rating,
         DENSE_RANK() OVER (ORDER BY t1.average_rating DESC) AS avg_rank
  FROM MOVIES M
  INNER JOIN t1 ON M.movieId = t1.movieId
),

t3 AS (
   SELECT t2.title, t2.movieId, t1.total_count,
         DENSE_RANK() OVER (ORDER BY total_count DESC) AS count_rank
  FROM t2
  INNER JOIN t1 ON t2.movieId = t1.movieId
)

SELECT t3.title AS count_title ,t3.total_count , t3.count_rank, t2.title AS avg_title, ROUND(t2.average_rating,2) average_rating, avg_rank FROM t3 INNER JOIN t2 ON t2.avg_rank = t3.count_rank
WHERE t2.avg_rank <= 10 AND t3.count_rank <= 10
"""

output = spark.sql(query)
output.show()

output.coalesce(1).write.mode("overwrite").format('csv').option('header', 'true') .option('delimiter', ',').save('/tmp/output_data/Movie_Data_Analysis/top_10_movies_in_average_and_number_of_ratings.csv')
print("Write Successful!")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+----------+--------------------+--------------+--------+
|         count_title|total_count|count_rank|           avg_title|average_rating|avg_rank|
+--------------------+-----------+----------+--------------------+--------------+--------+
|American Beauty (...|        204|         1|Boondock Saints, ...|          4.22|       1|
|Ace Ventura: Pet ...|        161|         2|       Brazil (1985)|          4.18|       2|
|    Mask, The (1994)|        157|         3|Cinema Paradiso (...|          4.16|       3|
|     Die Hard (1988)|        145|         4|       Snatch (2000)|          4.16|       4|
|Die Hard: With a ...|        144|         5|For a Few Dollars...|          4.15|       5|
|Groundhog Day (1993)|        143|         6|Lives of Others, ...|          4.12|       6|
|Dumb & Dumber (Du...|        133|         7|  Toy Story 3 (2010)|          4.11|       7|
|Monsters, Inc. (2...|        132|         8|Boogie Nights (1997)|          4.08|       8|

# 7. What is the average number of tags per movie in tagsDF? And the average number of tags per user? How does it compare with the average number of tags a user assigns to a movie?

In [44]:
query = """with t1 as(
          Select '1' as key, round((sum(CASE when tag IS NOT NULL THEN 1 ELSE 0 END)/count(distinct movieid)),2) as tags_per_movie
          from TAGS),

          t2 as ( Select '1' as key, (sum(CASE WHEN tag IS NOT NULL THEN 1 ELSE 0 END)/count(distinct userid)) as tags_per_user
          from TAGS)

          Select t1.tags_per_movie,t2.tags_per_user,
          CASE WHEN tags_per_user>tags_per_movie THEN 'tags_per_user is higher'
          ELSE 'tags_per_movie is higher' END as Comparison
          from t1 inner join t2 on t1.key=t2.key"""

output = spark.sql(query)
output.show()

output.coalesce(1).write.mode("overwrite").format('csv').option('header', 'true') .option('delimiter', ',').save('/tmp/output_data/Movie_Data_Analysis/tags_per_user_vs_tags_per_movie.csv')
print("Write Successful!")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+-------------+--------------------+
|tags_per_movie|tags_per_user|          Comparison|
+--------------+-------------+--------------------+
|          2.34|         63.5|tags_per_user is ...|
+--------------+-------------+--------------------+

Write Successful!

# 8. Identify the users that tagged movies without rating them

In [45]:
query = """WITH t1 AS
(SELECT DISTINCT T.userId FROM TAGS T
LEFT OUTER JOIN RATINGS R
ON T.movieId = R.movieId
WHERE R.userId IS NULL)
SELECT * FROM t1
"""

output = spark.sql(query)
output.show()

output.coalesce(1).write.mode("overwrite").format('csv').option('header', 'true') .option('delimiter', ',').save('/tmp/output_data/Movie_Data_Analysis/users_that_tagged_movies_without_rating_them.csv')
print("Write Successful!")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+
|userId|
+------+
|   474|
|   288|
|   543|
|   318|
+------+

Write Successful!

# 9. What is the average number of ratings per user in ratings DF? And the average number of ratings per movie?

In [46]:
query= """with t1 as(
          Select '1' as key, round((SUM(CASE when rating IS NOT NULL THEN 1 ELSE 0 END)/count(distinct userid)),2) as ratings_per_user
          from RATINGS),

          t2 as ( Select '1' as key, round((sum(CASE WHEN rating IS NOT NULL THEN 1 ELSE 0 END)/count(distinct movieid)),2) as ratings_per_movie
          from RATINGS)

          Select t1.ratings_per_user,t2.ratings_per_movie
          from t1 inner join t2 on t1.key=t2.key"""

output = spark.sql(query)
output.show()

output.coalesce(1).write.mode("overwrite").format('csv').option('header', 'true') .option('delimiter', ',').save('/tmp/output_data/Movie_Data_Analysis/avg_number_of_ratings_per_user_vs_per_movie.csv')
print("Write Successful!")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-----------------+
|ratings_per_user|ratings_per_movie|
+----------------+-----------------+
|           165.3|            10.37|
+----------------+-----------------+

Write Successful!

# 10. What is the predominant (frequency based) genre per rating level?

In [47]:
query= """WITH t1 AS(
          SELECT M.genres, R.rating, COUNT(*) as counts,
          DENSE_RANK()OVER(PARTITION BY R.rating ORDER BY COUNT(*) DESC) AS ranks
          FROM RATINGS R
          LEFT OUTER JOIN MOVIES M
          ON R.movieID=M.movieID
          GROUP BY 1,2)

          SELECT genres, rating AS most_frequent_genre FROM t1
          WHERE ranks=1
          ORDER BY rating DESC
          """
output = spark.sql(query)
output.show(10)

output.coalesce(1).write.mode("overwrite").format('csv').option('header', 'true') .option('delimiter', ',').save('/tmp/output_data/Movie_Data_Analysis/predominant_genre_per_rating_level.csv')
print("Write Successful!")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-------------------+
|genres|most_frequent_genre|
+------+-------------------+
| Drama|                5.0|
| Drama|                4.5|
| Drama|                4.0|
|Comedy|                3.5|
|Comedy|                3.0|
|Comedy|                2.5|
|Comedy|                2.0|
|Comedy|                1.5|
|Comedy|                1.0|
|Comedy|                0.5|
+------+-------------------+

Write Successful!

# 11. What is the predominant tag per genre and the most tagged genres?

In [48]:
query= """WITH t1 AS(
          SELECT M.genres, T.tag, COUNT(*) as counts,
          DENSE_RANK()OVER(PARTITION BY M.genres ORDER BY COUNT(*) DESC) AS rankS
          FROM MOVIES AS m
          LEFT OUTER JOIN TAGS AS t
          ON T.movieID=M.movieID
          GROUP BY 1,2)

          SELECT genres,tag AS most_frequent_tag FROM t1
          WHERE ranks=1
          ORDER BY genres DESC
          """
output = spark.sql(query)
output.show(10)

output.coalesce(1).write.mode("overwrite").format('csv').option('header', 'true') .option('delimiter', ',').save('/tmp/output_data/Movie_Data_Analysis/predominant_tag_per_genre.csv')
print("Write Successful!")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------+
|              genres|most_frequent_tag|
+--------------------+-----------------+
|             Western|             NULL|
|                 War|             NULL|
|            Thriller|             NULL|
|Sci-Fi|Thriller|IMAX|             NULL|
|     Sci-Fi|Thriller|             NULL|
|         Sci-Fi|IMAX|           sci-fi|
|         Sci-Fi|IMAX|      time-travel|
|              Sci-Fi|             NULL|
|     Romance|Western|             NULL|
|         Romance|War|        Hemingway|
+--------------------+-----------------+
only showing top 10 rows

Write Successful!

# 12. What are the most predominant (popularity based) movies?

In [49]:
query= """WITH t1 AS(
          SELECT R.movieID, M.title, COUNT(DISTINCT R.userId) AS counts,
          DENSE_RANK()OVER(ORDER BY COUNT(DISTINCT R.userId) DESC) as ranks
          FROM RATINGS R
          LEFT OUTER JOIN MOVIES M
          ON R.movieId = M.movieId
          GROUP BY 1,2)

          SELECT title,counts FROM t1
          WHERE ranks <= 10
          """

output = spark.sql(query)
output.show()

output.coalesce(1).write.mode("overwrite").format('csv').option('header', 'true') .option('delimiter', ',').save('/tmp/output_data/Movie_Data_Analysis/predominant_popular_movies.csv')
print("Write Successful!")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------+
|               title|counts|
+--------------------+------+
| Forrest Gump (1994)|   329|
|Shawshank Redempt...|   317|
| Pulp Fiction (1994)|   307|
|Silence of the La...|   279|
|  Matrix, The (1999)|   278|
|Star Wars: Episod...|   251|
|Jurassic Park (1993)|   238|
|   Braveheart (1995)|   237|
|Terminator 2: Jud...|   224|
|Schindler's List ...|   220|
+--------------------+------+

Write Successful!

# 13. Top 10 movies in terms of average rating (provided more than 30 users reviewed them)

In [50]:
query= """WITH t1 AS(
          SELECT movieid,avg(rating) AS avg_rating,
          DENSE_RANK()OVER (ORDER BY AVG(rating) DESC) AS ranks
          FROM RATINGS
          GROUP BY 1
          HAVING COUNT(DISTINCT userId)>30)

          SELECT M.title, ROUND(t1.avg_rating,9) AS avg_rating,t1.ranks FROM t1
          LEFT OUTER JOIN MOVIES M
          ON t1.movieId=M.movieId
          where ranks<=10
          """

output = spark.sql(query)
output.show()

output.coalesce(1).write.mode("overwrite").format('csv').option('header', 'true') .option('delimiter', ',').save('/tmp/output_data/Movie_Data_Analysis/top_ten_movies_in_terms_of_avg_rating.csv')
print("Write Successful!")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+-----+
|               title| avg_rating|ranks|
+--------------------+-----------+-----+
|Shawshank Redempt...|4.429022082|    1|
|Lawrence of Arabi...|        4.3|    2|
|Godfather, The (1...|  4.2890625|    3|
|   Fight Club (1999)| 4.27293578|    4|
|Cool Hand Luke (1...|4.271929825|    5|
|Dr. Strangelove o...|4.268041237|    6|
|  Rear Window (1954)|4.261904762|    7|
|Godfather: Part I...|4.259689922|    8|
|Departed, The (2006)|4.252336449|    9|
|   Goodfellas (1990)|       4.25|   10|
+--------------------+-----------+-----+

Write Successful!